In [ ]:
from medloader.api import DatasetEngine
from medloader.app import config as config
from medloader.app.logger import init_logger

logger = init_logger()

cfg = config.Configuration()

engine = DatasetEngine(
    logger=logger,
    config=cfg,
).go(mode="config")


In [ ]:
batch_size = 8
from torch.utils.data import DataLoader

dl = DataLoader(
            dataset=engine[0],
            batch_size=batch_size,
            pin_memory=True,
            shuffle=False,
            #sampler=DistributedSampler(dataset)
        )

In [ ]:
dl


In [ ]:
gpu_id = 1
from tqdm import tqdm

with tqdm(dl) as pbar:
    for i, batch in enumerate(pbar):
        source = batch["image"].to(gpu_id)
        targets = batch["target"].to(gpu_id)
        print(batch["args"])


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pydicom as dicom
import numpy as np

In [ ]:
source.shape


In [ ]:

slic = 1400
plt.imshow(source.shape[0,0,:,:])


In [ ]:
labels = engine[0][slic]["target"][0,:,:]
plt.imshow(labels)
np.unique(labels, return_counts=True)

In [ ]:
patient_folder = "/dataset/3Dircadb1.18"
mask_folder = "MASKS_DICOM"
masks_paths = [
                os.path.join(
                   patient_folder, mask_folder, maskdir
                ) for maskdir in  os.listdir(
                    os.path.join(patient_folder, mask_folder)
                ) if "livertumor" in maskdir
           ]
masks_paths.append(os.path.join(patient_folder, mask_folder, "liver"))
print (masks_paths)

In [ ]:

scan_id = 50
image_name_prefix = "image_"
mask_name_prefix = "segmentation_"
patient_scan = dicom.dcmread(
            os.path.join(patient_folder+"/PATIENT_DICOM", f'{image_name_prefix}{scan_id}')
        ).pixel_array.astype(np.float32)

label_scan = np.zeros(patient_scan.shape)
for folder in masks_paths:
    label_slice = dicom.dcmread(
            os.path.join(folder, f'{image_name_prefix}{scan_id}')
        ).pixel_array.astype(np.float32)/255
    print(folder, label_slice.max(), label_slice.min())
    if "livertumor" in folder:
        label_slice *= 2

    label_scan += label_slice

In [ ]:
plt.imshow(label_scan)
os.path.join(folder, f'{mask_name_prefix}{scan_id}')

In [ ]:
from medloader.loaders.preprocess_utils.mask_edges import clip_data
PATCH_SIZE = 300
image, mask = clip_data(patient_scan, label_scan, PATCH_SIZE)

In [ ]:
plt.imshow(image)


In [ ]:
plt.imshow(label_slice*255)
np.unique(label_slice*255, return_counts=True)

In [ ]:
np.save(
    "test.npy",
    mask
)

In [ ]:
plt.imshow(np.load("/dataset/clipped/3Dircadb1.17/segmentation_106.npy").astype(np.int16)[:PATCH_SIZE, :PATCH_SIZE])

In [ ]:
mask[mask > 2] = 2
np.unique(mask, return_counts=True)

In [ ]:
plt.imshow(np.load('/dataset/clipped/3Dircadb1.17/segmentation_106.npy'))

In [ ]:
mask = np.load('/dataset/clipped/3Dircadb1.17/segmentation_106.npy')
mask.max()